In [1]:
from torch.autograd import Variable
import torch
import torch.nn as nn
import numpy as np
import torch.nn as nn
import torch.nn.functional as F

# Convolution PYTORCH

### Définition d'une convolution propre

In [46]:
class MyConv1d(nn.Module):
    """
    Performs 1D convolution on images with a constant filter.
    Attributes
    ----------
        kernel (torch.FloatTensor): size c*c*h*w filter
        mode                 (str): 'single' or 'batch'
        stride               (int): dilation factor
        padding                   : instance of CircularPadding or torch.nn.ReplicationPad2d
    """
    def __init__(self, kernel, mode, pad_type = 'replicate', padding=0, stride=1):
        """
        Parameters
        ----------
            gpu                  (str): gpu id
            kernel (torch.FloatTensor): convolution filter
            mode                 (str): indicates if the input is a single image of a batch of images
            pad_type             (str): padding type (default is 'circular')
            padding              (int): padding size (default is 0)
            stride               (int): dilation factor (default is 1)
        """
        super(MyConv1d, self).__init__()
        self.kernel   = nn.Parameter(kernel,requires_grad=False)   
        self.mode     = mode #'single' or 'batch'
        self.stride   = stride
        if padding==0:
            size_padding = int((kernel[0,0].size(0)-1)/2)
        else:
            size_padding = padding
        if pad_type == 'replicate':
            self.padding = nn.ReplicationPad1d(size_padding)
        if pad_type == 'reflect':
            self.padding = nn.ReflectionPad1d(size_padding)
        if pad_type == 'zero':
            self.padding = nn.ConstantPad1d(size_padding,0)
            
    def forward(self, x): 
        """
        Performs a 2-D circular convolution.
        Parameters
        ----------
            x (torch.FloatTensor): image(s), size n*c*h*w 
        Returns
        -------
            (torch.FloatTensor): result of the convolution, size n*c*h*w if mode='single', 
                                 size c*h*w if mode='batch'
        """
        if self.mode == 'single':
            return F.conv1d(self.padding(x.unsqueeze(0)), self.kernel, stride=self.stride).data[0]
        if self.mode == 'batch':
            return F.conv1d(self.padding(x.data), self.kernel, stride=self.stride)

### TEST

In [47]:
x = torch.randn(1,4)

In [48]:
x

tensor([[ 0.7648, -0.6076,  0.0464, -1.0189]])

In [49]:
kernel = np.ones((1,1,20))
kernel = torch.FloatTensor(kernel)

In [50]:
m = MyConv1d(kernel, 'single', pad_type = 'zero', padding=0, stride=1)

In [51]:
m(x)

tensor([[[-0.8153, -0.8153, -0.8153]]])

In [29]:
x.sum()

tensor(1.0767)

In [30]:
x[0,:6].sum()

tensor(1.0767)

In [31]:
x[0,1:].sum()

tensor(1.1086)

### Ma convolution

In [32]:
n=4
xi = np.linspace(0,1,n)
a=1
kernel = xi**(a-1)

In [33]:
xi

array([0.        , 0.33333333, 0.66666667, 1.        ])

In [34]:
k = np.zeros((n,n))
for i in range(n):
    k[i,:i+1] = kernel[:i+1]

In [39]:
k.T

tensor([[1., 1., 1., 1.],
        [0., 1., 1., 1.],
        [0., 0., 1., 1.],
        [0., 0., 0., 1.]])

In [43]:
k=torch.FloatTensor(k)

In [44]:
torch.matmul(x.unsqueeze(0),k)

tensor([[[-0.0319,  0.3385, -0.3518,  1.0767]]])

In [45]:
x

tensor([[-0.0319,  0.3704, -0.6903,  1.4285]])

In [52]:
class MyConv1d(nn.Module):
    """
    Performs 1D convolution with kernel
    Attributes
    ----------
        kernel (torch.FloatTensor): size c*c*h*w filter
        mode                 (str): 'single' or 'batch'
        stride               (int): dilation factor
        padding                   : instance of CircularPadding or torch.nn.ReplicationPad2d
    """
    def __init__(self, kernel, mode):
        """
        Parameters
        ----------
            gpu                  (str): gpu id
            kernel (torch.FloatTensor): convolution filter
            mode                 (str): indicates if the input is a single image of a batch of images
            pad_type             (str): padding type (default is 'circular')
            padding              (int): padding size (default is 0)
            stride               (int): dilation factor (default is 1)
        """
        super(MyConv1d, self).__init__()
        self.kernel   = nn.Parameter(kernel.T,requires_grad=False)   
        self.mode     = mode #'single' or 'batch'
            
    def forward(self, x): 
        """
        Performs convolution.
        Parameters
        ----------
            x (torch.FloatTensor): image(s), size n*c*h*w 
        Returns
        -------
            (torch.FloatTensor): result of the convolution, size n*c*h*w if mode='single', 
                                 size c*h*w if mode='batch'
        """
        if self.mode == 'single':
            return torch.matmul(x.unsqueeze(0),self.kernel).data[0]
        if self.mode == 'batch':
            return torch.matmul(x.data,self.kernel)

In [53]:
 m = MyConv1d(torch.FloatTensor(k),'single')

In [54]:
m(x)

tensor([[-0.8153, -1.5801, -0.9725, -1.0189]])

In [55]:
xm = torch.randn((10,1,4))

In [56]:
mm = MyConv1d(torch.FloatTensor(k),'batch')

In [57]:
mm(xm)

tensor([[[ 1.7100,  1.3059,  1.8033, -0.3682]],

        [[-0.3770, -1.4885,  0.1213, -0.3180]],

        [[-0.1910, -1.3440, -1.5387, -2.0038]],

        [[-1.5182, -1.3060,  0.1131, -1.1023]],

        [[ 1.0715,  0.8020, -0.0766,  0.3836]],

        [[ 3.2560,  1.9025,  2.8441,  1.2860]],

        [[-3.5118, -3.5939, -2.5407, -0.4437]],

        [[-1.0441, -1.4403, -1.7088, -1.2483]],

        [[-0.4146, -0.0804, -0.3606,  0.3237]],

        [[-3.3132, -1.2709,  0.1289, -1.7600]]])

In [58]:
xm.data

tensor([[[ 0.4041, -0.4975,  2.1715, -0.3682]],

        [[ 1.1115, -1.6097,  0.4393, -0.3180]],

        [[ 1.1530,  0.1947,  0.4651, -2.0038]],

        [[-0.2122, -1.4191,  1.2154, -1.1023]],

        [[ 0.2694,  0.8786, -0.4602,  0.3836]],

        [[ 1.3535, -0.9415,  1.5580,  1.2860]],

        [[ 0.0821, -1.0533, -2.0970, -0.4437]],

        [[ 0.3962,  0.2685, -0.4605, -1.2483]],

        [[-0.3343,  0.2802, -0.6842,  0.3237]],

        [[-2.0423, -1.3998,  1.8888, -1.7600]]])

In [60]:
xm.size()

torch.Size([10, 1, 4])

In [61]:
xm.data.size()

torch.Size([10, 1, 4])